In [ ]:
import os
import sys
import time
import pickle

sys.path.append("../src")

import torch
from torch import nn

import torchvision
import torchvision.transforms as transforms


from baseline_model import BaselineModel
from enhanced_model import EnhancedModel
from helper_functions import run_model
from plots_functions import (
    multiple_runs_with_uncertainty_band,
    multiple_runs_with_every_run,
)


device = "cuda" if torch.cuda.is_available() else "cpu"


data_loader_args = {
    "batch_size": 64,
    "shuffle": True,
    "num_workers": 6,
    "pin_memory": True,
    "persistent_workers": True,
}


seed = 213
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
# initial setup
metadata = """all runs have 30 epochs, learning rate 0.01, each combination was run 5 times, 
4 dropout rates were tested [0, 0.25, 0.5, 0.75], and 5 L2 regularization parameters [0, 0.0001, 0.001, 0.01, 0.05], 
sgd optimizer with momentum 0.9 and Baseline model with 15 hidden units"""
epochs = 30
lr = 0.01
num_of_repeats = 5
dropouts = [0, 0.25, 0.5, 0.75]
regularizations = [0, 0.0001, 0.001, 0.01, 0.05]
experiment_dict = {
    "experiment name": "regularization",
    "experiments metadata": metadata,
}
if "regularization.pkl" not in os.listdir("../results"):
    pickle.dump(experiment_dict, open("../results/regularization.pkl", "wb"))
else:
    raise FileExistsError("File already exists, this notebook does not i")

# time and progress
start_time = time.time()
total_number_of_runs = num_of_repeats * len(regularizations) * len(dropouts)
number_of_completed_runs = 0

cinic_directory = "../data"
cinic_train = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(
        cinic_directory + "/train", transform=transforms.ToTensor()
    ),
    **data_loader_args,
)

cinic_test = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(
        cinic_directory + "/test", transform=transforms.ToTensor()
    ),
    **data_loader_args,
)

for dropout in dropouts:
    for regularization in regularizations:
        experiment_dict = pickle.load(open("../results/regularization.pkl", "rb"))
        metrics_list = []
        total_time_list = []
        for i in range(num_of_repeats):
            model = BaselineModel(
                input_shape=3, hidden_units=15, output_shape=10, dropout_p=dropout
            ).to(device)

            loss_fn = nn.CrossEntropyLoss()
            optimizer = torch.optim.SGD(
                params=model.parameters(),
                lr=lr,
                momentum=0.9,
                weight_decay=regularization,
            )

            metrics, total_time = run_model(
                cinic_train,
                cinic_test,
                model,
                loss_fn,
                optimizer,
                device=device,
                epochs=epochs,
            )

            metrics_list.append(metrics)
            total_time_list.append(total_time)

        experiment_dict[(dropout, regularization)] = metrics_list
        pickle.dump(experiment_dict, open("../results/regularization.pkl", "wb"))
        number_of_completed_runs += 1
        seconds = time.time() - start_time
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        print(
            f"{number_of_completed_runs}/{total_number_of_runs} ({round(100* number_of_completed_runs/total_number_of_runs)}%) done, runs {hours}h:{minutes}min"
        )